In [1]:
#imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import popsims
from popsims.simulator import Population, pop_mags
from popsims.galaxy import Disk, Halo
from popsims.plot_style import  plot_style
from astropy.coordinates import SkyCoord
import astropy.units as u
from tqdm import tqdm

plot_style()
#plt.style.use('dark_background')


import warnings
warnings.filterwarnings("ignore")

In [2]:
def compute_vols_and_numbers(df, gmodel, sptgrid, footprint, maglimits):
    from popsims.abs_mag_relations import POLYNOMIALS
    counts={}
    vols={}
    dists={}

    for spt in sptgrid:
        
        dmins=[]
        dmaxs=[]
        
        dmins_sd=[]
        dmaxs_sd=[]
        
        for k in maglimits.keys():
            mag_cut= maglimits[k]
            absmag= np.poly1d(POLYNOMIALS['absmags_spt']['dwarfs'][k]['fit'])(spt)
            
            #absmag_sd= np.poly1d(POLYNOMIALS['absmags_spt']['subdwarfs'][k]['fit'])(spt)
        
            mag_cut= maglimits[k]
            
            dmin=10.**(-(absmag-mag_cut[0])/5. + 1.)
            dmax=10.**(-(absmag-mag_cut[1])/5. + 1.)
            
            #dmin_sd=10.**(-(absmag_sd-14)/5. + 1.)
            #dmax_sd=10.**(-(absmag_sd-mag_cut)/5. + 1.)
        
            
            dmins.append(dmin)
            dmaxs.append(dmax)
            
            #dmins_sd.append(dmin)
            #dmaxs_sd.append(dmax)
            
        dmin=np.nanmedian(dmins)
        dmax=np.nanmedian(dmaxs)
        
        #dmin_sd=np.nanmedian(dmins_sd)
        #dmax_sd=np.nanmedian(dmaxs_sd)
        
        #print (spt, dmin, dmax)
        
        scale=[df.scale.mean(), df.scale_unc.mean(), df.scale_times_model.mean()]
        
        sn= len(df)
        #sn= len(df.query('population == "thin disk"'))
        #snt= len(df.query('population == "thick disk"'))
        #snh= len(df.query('population == "halo"'))
      
        sn_c= len(df.query('spt < {}'.format(spt, spt+0.9)))
        #snt_c= len(df.query('population == "thick disk" and spt >= {} and spt < {}'.format(spt, spt+0.9)))
        #snh_c= len(df.query('population == "halo" and spt >= {} and spt < {}'.format(spt, spt+0.9)))
        
        
        volumes={'volume': 0.0}
        
        cnts={'number':  sn_c*np.divide(scale[-1], sn)}
        for s in  footprint:
            l=s.galactic.l.radian
            b=s.galactic.b.radian
            volumes['volume'] += gmodel.volume(l, b, dmin, dmax)/len(footprint)
            #volumes['thick'] += tdisk.volume(l, b, dmin, dmax)/len(footprint)
            #volumes['halo'] += halo.volume(l, b, dmin, dmax)/len(footprint)
            
        vols.update({spt: volumes})
        counts.update({spt: cnts})
        dists.update({spt: dmax})
        
        
    return pd.DataFrame.from_records(vols).T.replace(np.inf, np.nan),\
    pd.DataFrame.from_records(counts).T.replace(np.inf, np.nan),\
    dists


In [3]:
footprint = SkyCoord(l=np.random.uniform(-np.pi, np.pi, 10)*u.rad, 
                     b= np.random.uniform(-np.pi/2, np.pi/2, 10)*u.rad, frame='galactic' )

In [6]:
#kirkpatrick
p_disk=Population(evolmodel= 'burrows1997',
              imf_power=-0.6,
              binary_fraction=0.2,
              age_range=[0.01, 8],
              mass_range=[0.01, .1],
             nsample=1e5)
    
p_disk.simulate()
disk= Disk()
p_disk.add_distances(disk, footprint.l.radian,footprint.b.radian, 0.1,  50_000, dsteps=5000)

In [ ]:
#kirkpatrick
p_tdisk=Population(evolmodel= 'burrows1997',
              imf_power=-0.6,
              binary_fraction=0.2,
              age_range=[0.01, 8],
              mass_range=[0.01, .1],
             nsample=1e6)
    
p_tdisk.simulate()
disk= Disk(L=3600, H=3600)
p_tdisk.add_distances(disk, footprint.l.radian,footprint.b.radian, 0.1,  50_000, dsteps=5000)

In [ ]:
#kirkpatrick
p1=Population(evolmodel= 'burrows1997',
              imf_power=-0.6,
              binary_fraction=0.2,
              age_range=[8, 14],
              mass_range=[0.01, .1],
             nsample=1e6)
    
p1.simulate()
p1.add_distances(Halo(), footprint.l.radian,footprint.b.radian, 0.1,  50_000, dsteps=5000)

#add magnitudes from pre-defined filters or pre-define polynomial cofficients
p1.add_magnitudes([ 'LSST_R', 'LSST_I', 'LSST_Z'], get_from='spt',object_type='subdwarfs')

In [ ]:
#NIRISS_F115W

In [ ]:
#kirkpatrick
p2=Population(evolmodel= 'burrows1997',
              imf_power=-2.5,
              binary_fraction=0.2,
              age_range=[8, 14],
              mass_range=[0.01, .1],
             nsample=1e6)
    
p2.simulate()
p2.add_distances(Halo(),footprint.l.radian,footprint.b.radian, 0.1, 50_000, dsteps=5000)

#add magnitudes from pre-defined filters or pre-define polynomial cofficients
p2.add_magnitudes([ 'LSST_R', 'LSST_I', 'LSST_Z'], get_from='spt', object_type='subdwarfs')

In [ ]:
#kirkpatrick
p3=Population(evolmodel= 'burrows1997',
              imf_power=2.5,
              binary_fraction=0.2,
              age_range=[8, 14],
              mass_range=[0.01, .1],
             nsample=1e6)
    
p3.simulate()
p3.add_distances(Halo(),  footprint.l.radian,footprint.b.radian, 0.1,  50_000, dsteps=5000)

#add magnitudes from pre-defined filters or pre-define polynomial cofficients
p3.add_magnitudes(['LSST_G', 'LSST_R', 'LSST_I', 'LSST_Z',  'NIRISS_F115W'], get_from='spt')

In [ ]:
cols=['mass', 'age', 'temperature', 'luminosity', 'is_binary', 'spt', \
      'prim_spt', 'sec_spt', 'pri_mass', 'sec_mass', 'prim_luminosity',\
      'sec_luminosity', 'scale', 'scale_unc', 'scale_times_model', 'l', 'b', \
      'distance',  'abs_LSST_R', 'abs_LSST_I', 'abs_LSST_Z', 'LSST_R', 'LSST_I', 'LSST_Z']

maglimits={'LSST_Z': [10, 26.1],  'LSST_R':[10, 27.5], 'LSST_I': [10, 26.8]}
maglimits1hr= {'LSST_Z': [10, 23.3], 'LSST_R':[10, 24.0], 'LSST_I': [10, 24.7]}
sptgrid=np.arange(15, 40)

In [ ]:
df1=p1.to_dataframe(cols)
df2=p2.to_dataframe(cols)
df3=p3.to_dataframe(cols)

In [ ]:
AREA=(18_000*u.degree**2).to(u.radian**2)

In [ ]:
n1=  compute_vols_and_numbers(df1, Halo(), sptgrid, footprint, maglimits)
n2=  compute_vols_and_numbers(df2, Halo(), sptgrid, footprint, maglimits)
n3=  compute_vols_and_numbers(df3, Halo(), sptgrid, footprint, maglimits)

In [ ]:
n1_1hr=  compute_vols_and_numbers(df1, Halo(), sptgrid, footprint, maglimits1hr)
n2_1hr=  compute_vols_and_numbers(df2, Halo(), sptgrid, footprint, maglimits1hr)
n3_1hr=  compute_vols_and_numbers(df3, Halo(), sptgrid, footprint, maglimits1hr)

In [ ]:
#kirkpatrick
p1_thin=Population(evolmodel= 'burrows1997',
              imf_power=-0.6,
              binary_fraction=0.2,
              age_range=[0.1, 8],
              mass_range=[0.01, .1],
             nsample=1e6)

#kirkpatrick
p1_thick=Population(evolmodel= 'burrows1997',
              imf_power=-0.6,
              binary_fraction=0.2,
              age_range=[8, 14],
              mass_range=[0.01, .1],
             nsample=1e6)

p1_thin.simulate()
p1_thick.simulate()



In [ ]:
p1_thin.add_distances(Disk(),  footprint.l.radian,footprint.b.radian, 0.1,  50_000, dsteps=5000)
p1_thick.add_distances(Disk(L=2600, H=1000),  footprint.l.radian,footprint.b.radian, 0.1,  50_000, dsteps=5000)

In [ ]:
p1_thin.add_magnitudes(['LSST_G', 'LSST_R', 'LSST_I', 'LSST_Z',  'NIRISS_F115W'], get_from='spt')
p1_thick.add_magnitudes(['LSST_R', 'LSST_I', 'LSST_Z'], get_from='spt',  object_type='subdwarfs')

In [ ]:
df1_thin=p1_thin.to_dataframe(cols)
df1_thick=p1_thick.to_dataframe(cols)

In [ ]:
n1_thin=  compute_vols_and_numbers(df1_thin, Disk(), sptgrid, footprint, maglimits)
n1_thick=  compute_vols_and_numbers(df1_thick, Disk(H=1000, L=3600), sptgrid, footprint, maglimits)

In [ ]:
fig, ax=plt.subplots(figsize=(7*0.7, 6*0.7))

plt.step(sptgrid, 0.0025*n1[0].values.flatten()*(n1[1].values.flatten()*AREA.value),\
         label=r'Halo', color='#0074D9', linewidth=3)

plt.step(sptgrid, n1_thin[0].values.flatten()*(n1_thin[1].values.flatten()*AREA.value),\
         label=r'Thin Disk', color='#FFFFFF', linewidth=3)

plt.step(sptgrid, 0.12*n1_thick[0].values.flatten()*(n1_thick[1].values.flatten()*AREA.value),\
         label=r'Thick Disk', color='#FF851B', linewidth=3)

ax.legend()
ax.set(yscale='log', xlabel='Spectral Type', ylabel='Number of Sources',  xticks=[15, 20, 25, 30, 35, 40],
       xticklabels=['M5', 'L0', 'L5', 'T0', 'T5', 'Y0'],  ylim=[1, 1e7])
ax.minorticks_on()
plt.savefig('../figures_lsst_counts.pdf', bbox_inches='tight')

In [ ]:
n1_1hr=  compute_vols_and_numbers(df1, Halo(), sptgrid, footprint, maglimits1hr)
n1_thin_1hr=  compute_vols_and_numbers(df1_thin, Disk(), sptgrid, footprint, maglimits1hr)
n1_thick_1hr=  compute_vols_and_numbers(df1_thick, Disk(H=1000, L=3600), sptgrid, footprint,maglimits1hr)

In [ ]:
fig, ax=plt.subplots(figsize=(7*0.7, 6*0.7))

plt.step(sptgrid, 0.0025*n1_1hr[0].values.flatten()*(n1_1hr[1].values.flatten()*AREA.value),\
         label=r'Halo', color='#0074D9', linewidth=3)

plt.step(sptgrid, n1_thin_1hr[0].values.flatten()*(n1_thin_1hr[1].values.flatten()*AREA.value),\
         label=r'Thin Disk', color='#FFFFFF', linewidth=3)

plt.step(sptgrid, 0.12*n1_thick_1hr[0].values.flatten()*(n1_thick_1hr[1].values.flatten()*AREA.value),\
         label=r'Thick Disk', color='#FF851B', linewidth=3)

ax.legend()
ax.set(yscale='log', xlabel='Spectral Type', ylabel='Number of Sources',  xticks=[15, 20, 25, 30, 35, 40],
       xticklabels=['M5', 'L0', 'L5', 'T0', 'T5', 'Y0'], ylim=[1, 1e7])
ax.minorticks_on()
plt.savefig('../figures_lsst_counts_1hr.pdf', bbox_inches='tight')

In [ ]:
dp0_data=pd.read_csv('/Volumes/lacie/popsimsdata/cone_search.csv')


In [ ]:
fig, ax=plt.subplots(figsize=(10, 6))




for x, l in zip([df1_thin, df1_thick], ['Thin Disk', 'Thick Disk']):
    x['r-i']=x.LSST_R-x.LSST_I
    x['i-z']=x.LSST_I-x.LSST_Z
    
    for k in maglimits.keys():
            mag_cut= maglimits[k]
            x= (x[x[k]< mag_cut[-1]]).reset_index(drop=True)
            
    ax.scatter(x['i-z'], x['r-i'], label=l, s=.1)


ax.scatter(dp0_data.mag_i-dp0_data.mag_z, dp0_data.mag_r-dp0_data.mag_i,  label='DP0 Data', s=.1, c='k')


ax.set(xlabel='i-z', ylabel='r-i', xlim=[-0.5, 3])
ax.legend(markerscale=20)
ax.minorticks_on()
plt.savefig('../figures_lsst_colors.jpeg', bbox_inches='tight')

In [ ]:
#hack to get get g-r
from popsims.abs_mag_relations import POLYNOMIALS
po= POLYNOMIALS['colors_{}'.format('spt')]['subdwarfs']

po_g=POLYNOMIALS['absmags_{}'.format('spt')]['dwarfs']

In [ ]:
def get_values(x, pol, k):
    fit=np.poly1d(pol[k]['fit'])
    rng=pol[k]['range']
    mag_key=pol[k]['y']
    offset=pol[k]['x0']
    #put constraints on spt range
    mask= np.logical_and(x >rng[0], x <=rng[-1])
    abs_mag=np.random.normal(fit(x-offset),0.1)
    masked_abs_mag= np.ma.masked_array(data=abs_mag, mask=~mask)
    return masked_abs_mag.filled(np.nan)

In [ ]:
gs= get_values(df1_thin.spt.values.astype(float), po_g, 'LSST_G')

In [ ]:
fig, ax=plt.subplots()
ax.scatter(df1_thin.spt.values.astype(float), gs)

In [ ]:
df1_thin['g-r']=gs-df1_thin.abs_LSST_R.values

In [ ]:
gr= get_values(df1_thick.spt.values.astype(float), po, 'LSST_GandLSST_R')

In [ ]:
df1_thick['g-r']= gr

In [ ]:
fig, ax=plt.subplots(figsize=(10, 6))


ax.scatter(dp0_data.mag_r-dp0_data.mag_z, dp0_data.mag_g-dp0_data.mag_r,  label='DP0 Data', s=.1, color='k')

for x, l in zip([df1_thin, df1_thick], ['Thin Disk', 'Thick Disk']):
    x['r-i']=x.LSST_R-x.LSST_I
    x['i-z']=x.LSST_I-x.LSST_Z
    x['r-z']=x.LSST_R-x.LSST_Z
    for k in maglimits.keys():
            mag_cut= maglimits[k]
            x= (x[x[k]< mag_cut[-1]]).reset_index(drop=True)
            
    ax.scatter(x['r-z'], x['g-r'], label=l, s=.1)






ax.set(xlabel='r-z', ylabel='g-r', ylim=[-0.5, 4], xlim=[-0.5, 6])
ax.legend(markerscale=20)
ax.minorticks_on()
plt.savefig('../figures_lsst_colors_gr.jpeg', bbox_inches='tight', facecolor='white')

In [ ]:
dp0_data['i-z']=dp0_data.mag_i-dp0_data.mag_z
dp0_data['r-i']=dp0_data.mag_r-dp0_data.mag_i
dp0_data['g-r']=dp0_data.mag_g-dp0_data.mag_r
dp0_data['r-z']=dp0_data.mag_r-dp0_data.mag_z

In [ ]:
for k in maglimits.keys():
    mag_cut= maglimits[k]
    df1_thin= (df1_thin[df1_thin[k]< mag_cut[-1]]).reset_index(drop=True)
    df1_thick= (df1_thick[df1_thick[k]< mag_cut[-1]]).reset_index(drop=True)

In [ ]:
#create machine learning data
data=(pd.concat([df1_thin, df1_thick, dp0_data])[['r-i', 'i-z', 'g-r', 'r-z']]).dropna().values

In [ ]:
data.shape

In [ ]:
data.shape

In [ ]:
import hdbscan
from sklearn.datasets import make_blobs

#data, _ = make_blobs(1000)

clusterer = hdbscan.HDBSCAN(min_cluster_size=3)
cluster_labels = clusterer.fit_predict(data)

In [ ]:

fig, ax=plt.subplots()
plt.scatter(data.T[0], data.T[1], s=.1, c=cluster_labels, cmap='cividis')


In [ ]:
=bo,y

In [ ]:
fig, ax=plt.subplots(figsize=(6*0.7, 8*0.7))
plt.step(sptgrid, 0.0025*n1[1].values.flatten()*(n1[1].values.flatten()*AREA.value),\
         label=r'$dn/dm \propto m^{-0.6} $', color='#0074D9', linewidth=3)
plt.step(sptgrid, 0.0025*n2[1].values.flatten()*(n2[1].values.flatten()*AREA.value), \
         label=r'$dn/dm \propto m^{-2.5} $', color='#FF851B', linestyle='-', linewidth=3)
plt.step(sptgrid, 0.0025*n3[1].values.flatten()*(n3[1].values.flatten()*AREA.value), \
         label=r'$dn/dm \propto m^{2.5} $', color='#111111', linestyle='-', linewidth=3)

#plt.step(sptgrid, 0.0025*n1_1hr[0].values.flatten()*n1_1hr[1].values.flatten()*AREA.value,\
#         label=r'$dn/dm \propto m^{-0.6} $', color='#0074D9', linestyle=':', linewidth=3)
#plt.step(sptgrid, 0.0025*n2_1hr[0].values.flatten()*n2_1hr[1].values.flatten()*AREA.value, \
#         label=r'$dn/dm \propto m^{-2.5} $', color='#FF851B', linestyle=':', linewidth=3)
#plt.step(sptgrid, 0.0025*n3_1hr[0].values.flatten()*n3_1hr[1].values.flatten()*AREA.value, \
#         label=r'$dn/dm \propto m^{2.5} $', color='#111111', linestyle=':', linewidth=3)

plt.legend()
ax.set(yscale='log', xlabel='Spectral Type', ylabel='Space Density of Halo Stars',  xticks=[15, 20, 25, 30, 35, 40],
       xticklabels=['M5', 'L0', 'L5', 'T0', 'T5', 'Y0'])
ax.minorticks_on()
plt.savefig('../figures_lsst_imfs.pdf', bbox_inches='tight')

In [ ]:
maglimits_jwst={'NIRISS_F115W': [10, 29.]}

n1_jw=  compute_vols_and_numbers(df1, Halo(), sptgrid, footprint, maglimits_jwst)
n2_jw=  compute_vols_and_numbers(df2, Halo(), sptgrid, footprint, maglimits_jwst)
n3_jw=  compute_vols_and_numbers(df3, Halo(), sptgrid, footprint, maglimits_jwst)

In [ ]:
AREA_JWST=(200*u.arcmin**2).to(u.radian**2)

In [ ]:
def cum_numbers(sptgrid, n, area):
    n=0.0025*n[0].values.flatten()*n[1].values.flatten()*area.value  
    return np.cumsum(n)

In [ ]:
cum_numbers(sptgrid, n1_jw, AREA_JWST)

In [ ]:
fig, ax=plt.subplots(figsize=(6*0.7, 8*0.7))

n=cum_numbers(sptgrid, n1_jw, AREA_JWST)
plt.step(sptgrid, n ,\
         label=r'$dn/dm \propto m^{-0.6} $', color='#0074D9', linewidth=3)
plt.fill_between(sptgrid, n-np.sqrt(n), n+np.sqrt(n),  color='#0074D9', step="pre", alpha=0.4)

n2=cum_numbers(sptgrid, n2_jw, AREA_JWST)
plt.step(sptgrid, n2,\
         label=r'$dn/dm \propto m^{-2.5} $', color='#FF851B', linestyle='-', linewidth=3)
plt.fill_between(sptgrid, n2-np.sqrt(n2), n2+np.sqrt(n2), color='#FF851B', step="pre", alpha=0.4)


n3=cum_numbers(sptgrid, n3_jw, AREA_JWST)
plt.step(sptgrid, cum_numbers(sptgrid, n3_jw, AREA_JWST),\
         label=r'$dn/dm \propto m^{2.5} $', color='#111111', linestyle='-', linewidth=3)
plt.fill_between(sptgrid, n3-np.sqrt(n3), n3+np.sqrt(n3), color='#111111', step="pre", alpha=0.4)


plt.legend()
ax.minorticks_on()

ax.set(yscale='log', xlabel='Spectral Type', ylabel='Cumulative Number of Halo Stars',  xticks=[15, 20, 25, 30, 35, 40],
       xticklabels=['M5', 'L0', 'L5', 'T0', 'T5', 'Y0'])
ax.minorticks_on()
plt.savefig('../figures_jwst_imfs.pdf', bbox_inches='tight')